# Multicast Client 1

In [16]:
def mostrar_pregunta(question, opciones):
    
    print("\nPregunta:\n", question)
    print("\nOpciones:")

    for i in range(len(opciones)):
        print("・", opciones[i])

In [17]:
#Bucle con recieve

In [18]:
import socket
import struct
import pickle
import threading

MCAST_GRP = '224.1.1.1'
MCAST_PORT = 5007
clientID = 'User 1'

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)

#Para unirse al grupo de multicast
sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

#Listen only on the socket
sock.bind(('', MCAST_PORT))

msg = 'Joined the Game'
data_pack = (clientID, msg)
packed = pickle.dumps(data_pack)
sock.sendto(packed, (MCAST_GRP, MCAST_PORT))

#Tell the OS to add the socket to the multicast group
mreq = struct.pack("4sl", socket.inet_aton(MCAST_GRP), socket.INADDR_ANY)
sock.setsockopt(socket.IPPROTO_IP, socket.IP_ADD_MEMBERSHIP, mreq)

#Set TTL
ttl = struct.pack('b', 1)
sock.setsockopt(socket.IPPROTO_IP, socket.IP_MULTICAST_TTL, ttl)



quit = False

def recieve_function():
    global quit 
    while quit==False:
    
        try:
            packed = sock.recv(1024)
            data = pickle.loads(packed)

            if (str(data[1]) == "SERVER-QUESTION"):

                mostrar_pregunta(str(data[2]), data[3])


            if (str(data[1]) == 'END'):
                #Game ended, server OFF

                quit = True
                sock.close()
            
            
            print(str(data[0]) + ": " + str(data[1]))
            
        except(OSError):
            print("Socket closed")
            break
            
    print("FIN DEL JUEGO")

#Thread para recibir mensajes del server
reader = threading.Thread(target=recieve_function)
reader.start()

sock.sendto(pickle.dumps((clientID, "JOIN")) , (MCAST_GRP, MCAST_PORT))

while quit==False:
    msg = input("Introduce tu respuesta a la pregunta (Utiliza 'Quit' para salir en cualquier momento): ")
    
    try:
        #Server will take action upon recieving a Quit message from a certain User
        data_pack = (clientID, msg)
        packed = pickle.dumps(data_pack)
        sock.sendto(packed, (MCAST_GRP, MCAST_PORT))

        if (msg == "Quit"):

            sock.close()
            quit = True
            
    except(OSError):
        print("Socket closed")
        break

User 1: JOIN
GameServer: User 1 joined the game.
GameServer: Una nueva partida esta a punto de empezar!

Pregunta:
 Which character class in Dungeons and Dragons 5th edition gains it&#039;s powers from making a pact with a being of higher power?

Opciones:
・ Wizard
・ Sorceror
・ Cleric
・ Warlock
GameServer: SERVER-QUESTION
Introduce tu respuesta a la pregunta (Utiliza 'Quit' para salir en cualquier momento): Wizard
User 1: Wizard
GameServer: El usuario User 1 envió una respuesta incorrecta, la respuesta correcta era: Warlock
Ranking:
#1 Usuario: User 1 Puntos: 0


Pregunta:
 Gabe Newell was born in which year?

Opciones:
・ 1970
・ 1960
・ 1972
・ 1962 
GameServer: SERVER-QUESTION
Introduce tu respuesta a la pregunta (Utiliza 'Quit' para salir en cualquier momento): 1962 
User 1: 1962 
GameServer: User 1 envio la respuesta correcta, +1 punto
Ranking:
#1 Usuario: User 1 Puntos: 1


Pregunta:
 Leonardo da Vinci&#039;s Mona Lisa does not have eyebrows.

Opciones:
・ False
・ True
GameServer: SER